In [1]:
# pre-main imports: 
import os
import itertools
import functools
import logging
import pipeop
from logdecorator import log_on_start, log_on_end, log_on_error, log_exception
import wrapt
from fabric import Connection
from tqdm.notebook import tqdm #from tqdm import tqdm if not in notebook

In [2]:
# main-imports: custom decorators and functions
import shared_variables
shared_variables.enabled_wrapt_decorators = True # apply or not custom core decorators
from core_decorators import *
from core_functions import *

In [3]:
import integration_curator

### project specific code

In [4]:
subset_generator

<function core_functions.subset_generator(a_generator)>

### git save notebook

In [ ]:
# upload notebook to gitlab/github:

to_save = '*.ipynb *.py'
#current_dir=os.getcwd()
git_save_dir=os.getcwd()

c = Connection('mercury@hgi-farm5')
with c.cd(git_save_dir):
    # github repo requires using the correct deploy key:
    c.run('git config remote.origin.url git@gn5.githostname_github_bsub_jupyter_lab:wtsi-hgi/bsub_jupyter_lab.git')
    c.run('git config --local user.name gn5')
    c.run('git config --local user.email gn5@sanger.ac.uk')
    # now can pull:
    c.run('git pull')
    # c.run('cp ' + current_dir+'/'+nb_name + ' ' + git_save_dir+'/')
    #c.run('ls -ltra')

In [ ]:
with c.cd(git_save_dir):
    # now can add, commit and push files:
    c.run('git add ' + to_save)
    c.run('git commit -m auto || true')
    c.run('git push -u origin')

#### how ssh key works:

the `gn5.githostname_github_bsub_jupyter_lab` hostname serves my ssh key because:
```
cat /nfs/users/nfs_m/mercury/.ssh/config
Host gn5.githostname_github_bsub_jupyter_lab # used by gn5 as deploy key for github repo wtsi-hgi/bsub_jupyter_lab
    AddKeysToAgent yes                                                                              
    Hostname github.com                                                                             
    User git                                                                                        
    IdentityFile ~/.ssh/id_rsa_gn5_github_bsub_jupyter_lab            
...    
```

### install genestack/api_clients.zip lib from farm5-head1

In [ ]:
c_head1 = Connection('mercury@farm5-head1')
with c_head1.cd('/lustre/scratch118/humgen/resources/conda_envs/nextflow20_libs'): # ssh to farm5-head1 with fabric Connection()
    #c_head1.run('ls -ltra')
    c_head1.run('hostname')
    with c_head1.prefix('conda activate nextflow20'): # activate conda env with fabric prefix()
        c_head1.run('conda info --envs') # to check that correct conda env is loaded
        c_head1.run('which pip')
        c_head1.run('which python')
        c_head1.run('pip --version')
        c_head1.run('python --version')

        # install genestack api_clients.zip
        c_head1.run('cp /software/hgi/installs/genestack/api_clients.zip .')
        c_head1.run('unzip api_clients.zip -d .') 
        c_head1.run('pip install backports.functools-lru-cache')
        # c_head1.run('pip install genestack-client') ## only available for python 2, it's being ported to python3
        for entry in os.scandir('/lustre/scratch118/humgen/resources/conda_envs/nextflow20_libs/python'):
            if entry.is_dir(follow_symlinks=False):
                print(entry.path)
                c_head1.run('pip install -e "' + entry.path + '"')

In [8]:
# check that install worked: shutdown notebook, resubmit bsub command and
import integration_curator